In [302]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

def splitSpreads(tag):
    if tag.contents[2][0] == '-':
        spread = float(tag.contents[2].split( )[0].replace('½','.5'))
        return [spread, -spread]
    else:
        spread = float(tag.contents[4].split( )[0].replace('½','.5'))
        return [-spread, spread]

def splitLines(lines):
    if len(lines) > 0:
        if lines[0] == '-':
            try: return [int(l) for l in lines.split('+')]
            except: return [int('-'+l) for l in lines.split('-')[1:]]
        elif lines[0] == '+':
            return [int(lines.split('-')[0]), int('-'+lines.split('-')[1])]
    return [0,0]


def spread2odds(spread):
    return -0.0303*spread+0.5

def line2odds(line):
    if line < 0:
        return line/(line-100)
    else:
        return line/(line+100)



url = 'http://www.vegasinsider.com/college-football/odds/las-vegas/'
soup = bs(requests.get(url).text, 'html.parser')

teams = [t.text.strip() for t in soup.find_all('a', {'class':'tabletext'})]
games = [[teams[i], teams[i+1]] for i in range(0, len(teams), 2)]

tags = [tag.contents[1] for tag in soup.find_all('td', {'width':'56'})]
spreads = []


for i in range(0,len(tags),9):
    try: spreads.append(splitSpreads(tags[i]))
    except: spreads.append(splitSpreads(tags[i+1]))


url = 'http://www.vegasinsider.com/college-football/odds/las-vegas/money/'
soup = bs(requests.get(url).text, 'html.parser')
lines = [splitLines(l.text.strip()) for l in soup.find_all('td', {'width':'56'})[1::9]]


spreadOdds = [[spread2odds(s[0]), spread2odds(s[1])] for s in spreads]
lineOdds = [[line2odds(l[0]), line2odds(l[1])] for l in lines]
lineOdds = [[l[0]/sum(l), l[1]/sum(l)] for l in lineOdds]
odds = [[spreadOdds[i][0]*lineOdds[i][0], spreadOdds[i][1]*lineOdds[i][1]] for i in range(len(games))]

wTeams = []
wOdds = []
for i in range(len(games)):
    if odds[i][0] > odds[i][1]:
        wTeams.append(games[i][0])
        wOdds.append(odds[i][0])
    else:
        wTeams.append(games[i][1])
        wOdds.append(odds[i][1])
        
pd.DataFrame({'Team':wTeams, 'Odds':wOdds}).sort_values(by='Odds').reset_index(drop=True)

,Team,Odds
0,Kansas State,0.271437
1,Western Kentucky,0.288212
2,Tennessee,0.290958
3,James Madison,0.292487
4,Wisconsin,0.304792
5,Virginia Tech,0.305992
6,Mississippi State,0.312514
7,California,0.319474
8,Arizona State,0.328624
9,LSU,0.349824
